# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import sqlalchemy
from sqlalchemy import create_engine

import nltk
nltk.download(['punkt','wordnet'])
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

import re
import numpy as np
import pandas as pd

import pickle

from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier 
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import classification_report

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [2]:
# load data from database
# engine = create_engine('sqlite:///InsertDatabaseName.db')
engine = create_engine('sqlite:///disaster_response.db')
df = pd.read_sql("SELECT * FROM disaster_categories", engine)
X = df.message.values
y = df.iloc[:, 4:].values

### 2. Write a tokenization function to process your text data

In [3]:
def tokenize(text):
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()
    clean_tokens = []
    
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [4]:
pipeline = Pipeline([
    ('features', FeatureUnion([

        ('text_pipeline', Pipeline([
            ('vect', CountVectorizer(tokenizer=tokenize)),
            ('tfidf', TfidfTransformer())
        ]))
    ])),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y)
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('features', FeatureUnion(n_jobs=1,
       transformer_list=[('text_pipeline', Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_d...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [7]:
y_pred = pipeline.predict(X_test)

In [8]:
pipeline.score(X_train, y_train)

0.81341687041564792

In [9]:
accuracy = (y_pred == y_test).mean()
print("Accuracy:", accuracy)

Accuracy: 0.942067736185


In [10]:
for i in range(0,36):
    print("Category: "+str(i+1), classification_report([row[i] for row in y_test], [row[i] for row in y_pred]))

Category: 1              precision    recall  f1-score   support

          0       0.62      0.37      0.46      1540
          1       0.82      0.93      0.87      4962
          2       0.75      0.14      0.24        43

avg / total       0.77      0.79      0.77      6545

Category: 2              precision    recall  f1-score   support

          0       0.88      0.98      0.93      5405
          1       0.83      0.39      0.53      1140

avg / total       0.88      0.88      0.86      6545

Category: 3              precision    recall  f1-score   support

          0       1.00      1.00      1.00      6521
          1       0.00      0.00      0.00        24

avg / total       0.99      1.00      0.99      6545

Category: 4              precision    recall  f1-score   support

          0       0.73      0.89      0.80      3857
          1       0.77      0.53      0.63      2688

avg / total       0.75      0.74      0.73      6545

Category: 5              precision    r

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Category: 19              precision    recall  f1-score   support

          0       0.87      1.00      0.93      5667
          1       0.49      0.03      0.05       878

avg / total       0.82      0.87      0.81      6545

Category: 20              precision    recall  f1-score   support

          0       0.93      1.00      0.97      6111
          1       0.00      0.00      0.00       434

avg / total       0.87      0.93      0.90      6545

Category: 21              precision    recall  f1-score   support

          0       0.96      1.00      0.98      6263
          1       0.62      0.07      0.13       282

avg / total       0.95      0.96      0.94      6545

Category: 22              precision    recall  f1-score   support

          0       0.95      1.00      0.98      6214
          1       0.76      0.09      0.16       331

avg / total       0.94      0.95      0.93      6545

Category: 23              precision    recall  f1-score   support

          0       0.9

### 6. Improve your model
Use grid search to find better parameters. 

In [11]:
MultiOutputClassifier(RandomForestClassifier()).get_params().keys()

dict_keys(['estimator__bootstrap', 'estimator__class_weight', 'estimator__criterion', 'estimator__max_depth', 'estimator__max_features', 'estimator__max_leaf_nodes', 'estimator__min_impurity_decrease', 'estimator__min_impurity_split', 'estimator__min_samples_leaf', 'estimator__min_samples_split', 'estimator__min_weight_fraction_leaf', 'estimator__n_estimators', 'estimator__n_jobs', 'estimator__oob_score', 'estimator__random_state', 'estimator__verbose', 'estimator__warm_start', 'estimator', 'n_jobs'])

In [12]:
parameters = {
        'clf__estimator__n_estimators': [100, 200],
        'clf__estimator__criterion': ['gini', 'entropy']
    }

cv = GridSearchCV(pipeline, param_grid=parameters)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [13]:
cv.fit(X_train, y_train)

GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('features', FeatureUnion(n_jobs=1,
       transformer_list=[('text_pipeline', Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_d...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'clf__estimator__n_estimators': [100, 200, 500], 'clf__estimator__criterion': ['gini', 'entropy']},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [14]:
cv.best_params_

{'clf__estimator__criterion': 'gini', 'clf__estimator__n_estimators': 200}

In [15]:
cv.score(X_train, y_train)

0.99933781581092096

In [16]:
y_pred = cv.predict(X_test)
accuracy = (y_pred == y_test).mean()
print("Accuracy:", accuracy)

Accuracy: 0.946307613955


In [17]:
for i in range(0,36):
    print("Category: "+str(i+1), classification_report([row[i] for row in y_test], [row[i] for row in y_pred]))

Category: 1              precision    recall  f1-score   support

          0       0.73      0.26      0.39      1540
          1       0.80      0.97      0.88      4962
          2       0.75      0.14      0.24        43

avg / total       0.79      0.80      0.76      6545

Category: 2              precision    recall  f1-score   support

          0       0.89      0.99      0.94      5405
          1       0.89      0.44      0.59      1140

avg / total       0.89      0.89      0.88      6545

Category: 3              precision    recall  f1-score   support

          0       1.00      1.00      1.00      6521
          1       0.00      0.00      0.00        24

avg / total       0.99      1.00      0.99      6545

Category: 4              precision    recall  f1-score   support

          0       0.78      0.89      0.83      3857
          1       0.80      0.63      0.70      2688

avg / total       0.79      0.78      0.78      6545

Category: 5              precision    r

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Category: 18              precision    recall  f1-score   support

          0       0.96      1.00      0.98      6229
          1       0.79      0.08      0.15       316

avg / total       0.95      0.95      0.94      6545

Category: 19              precision    recall  f1-score   support

          0       0.87      1.00      0.93      5667
          1       0.82      0.01      0.02       878

avg / total       0.86      0.87      0.81      6545

Category: 20              precision    recall  f1-score   support

          0       0.93      1.00      0.97      6111
          1       1.00      0.00      0.00       434

avg / total       0.94      0.93      0.90      6545

Category: 21              precision    recall  f1-score   support

          0       0.96      1.00      0.98      6263
          1       0.81      0.07      0.14       282

avg / total       0.95      0.96      0.94      6545

Category: 22              precision    recall  f1-score   support

          0       0.9

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [7]:
from sklearn.neighbors import KNeighborsRegressor

In [8]:
new_pipeline = Pipeline([
    ('features', FeatureUnion([
        ('text_pipeline', Pipeline([
            ('vect', CountVectorizer(tokenizer=tokenize)),
            ('tfidf', TfidfTransformer())
        ]))
    ])),
    ('clf', MultiOutputClassifier(KNeighborsRegressor()))
])

In [9]:
parameters = {
        'clf__estimator__weights': ['uniform', 'distance'],
        'clf__estimator__leaf_size': [30, 40]
    }

cv_KNN = GridSearchCV(new_pipeline, param_grid=parameters)

In [ ]:
cv_KNN.fit(X_train, y_train)

In [ ]:
cv_SVM.best_params_

In [ ]:
cv_SVM.score(X_train, y_train)

In [ ]:
y_pred = cv_SVM.predict(X_test)
accuracy = (y_pred == y_test).mean()
print("Accuracy:", accuracy)

In [ ]:
for i in range(0,36):
    print("Category: "+str(i+1), classification_report([row[i] for row in y_test], [row[i] for row in y_pred]))

### 9. Export your model as a pickle file

In [ ]:
pickle.dump(model, open(filename, 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.